# Memory Usage Estimation

Sometimes it's nice to know how much memory your index will take up before you create it. Redis retrieval optimizer provides a helper function that makes it easy to insert data into Redis and run memory usage to get the closest estimate possible.

## Inputs

- `sample_object`: A single object that represents the data you want to store in Redis. This object should contain all the fields you want to store, not including the vector field. The vector field does not need to be populated, as the function will generate a sample vector for you.
- `num_objects`: The number of objects you want to store in Redis. This is used to generate the sample data.
- `schema`: The schema for the index you want to create. This should be a `IndexSchema` object.
- `embedding_model_info`: A dictionary containing the information needed to create the embedding model. This should include the `type` of embedding model, the `model` name, and the `dtype` (data type) of the embeddings.
- `redis_url`: The URL for your Redis instance. This is optional and defaults to `redis://localhost:6379`.
- `vector_field_name`: The name of the vector field in your schema. This is optional and defaults to `vector`.

## Outputs

- `info_used_memory_mb`: Total memory currently used by the Redis server, in megabytes, taken from INFO MEMORY.
- `index_memory_mb`: Memory used by the index structures themselves (from total_index_memory_sz_mb), in megabytes.
- `object_memory_mb`: Approximate memory used by the stored documents/keys for this index, in megabytes.
- `total_memory_mb`: Sum of index_memory_mb and object_memory_mb; total memory footprint for this index + its data, in megabytes.
- `index_memory_gb`: Same as index_memory_mb, but converted to gigabytes.
- `object_memory_gb`: Same as object_memory_mb, but converted to gigabytes.
- `total_memory_gb`: Same as total_memory_mb, but converted to gigabytes.
- `info_used_memory_gb`: Same as info_used_memory_mb, but converted to gigabytes.
- `single_key_memory_mb`: Memory used by a single stored key/document, in megabytes (from MEMORY USAGE on one key).

In [4]:
## Setup your inputs

from redisvl.schema import IndexSchema

index_name = "size_test"

schema = IndexSchema.from_dict({
  "index": {
    "name": index_name,
    "prefix": index_name,
    "storage_type": "hash"
  },
  "fields": [
    {
        "name": "title",
        "type": "text",
    },
    {
        "name": "description",
        "type": "text",
    },
    {
        "name": "genre",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "rating",
        "type": "numeric",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "vector",
        "type": "vector",
        "attrs": {
            "dims": 384,
            "distance_metric": "cosine",
            "algorithm": "flat",
            "datatype": "float32"
        }
    }
  ]
})

## Sample object includes all fields except vector
sample_object = {
    "id": 1,
    "title": "Explosive Pursuit",
    "genre": "action",
    "rating": 7,
    "description": "A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse."
}

## Provide vector details and function will take care of invoking the model
embedding_model_info = {
    "type": "hf",
    "model": "sentence-transformers/all-MiniLM-L6-v2",
    "dtype": "float32"
}

num_objects = 1000


In [ ]:
from redis_retrieval_optimizer.memory_usage.utils import estimate_index_size

index_size = estimate_index_size(
    sample_object, 
    num_objects, 
    schema, 
    embedding_model_info, 
    redis_url="redis://localhost:6379" # need redis instance running
)

In [6]:
index_size

{'info_used_memory_mb': 5.9783782958984375,
 'index_memory_mb': 1.9206666946411133,
 'object_memory_mb': 2.2040557861328125,
 'total_memory_mb': 4.124722480773926,
 'index_memory_gb': 0.0018756510689854622,
 'object_memory_gb': 0.0021523982286453247,
 'total_memory_gb': 0.004028049297630787,
 'info_used_memory_gb': 0.005838260054588318,
 'single_key_memory_mb': 0.0021762847900390625}